<a href="https://colab.research.google.com/github/raghavkaushik2004/AlexNet-architechture/blob/main/alexnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from keras import datasets, layers, models, losses

In [4]:
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


Now, the dataset MNIST has the dimensions of 28x28, so for making the number of channels = 3, we repeat it 3 times to get 28x28x3. So we have to reshape it to 227x227x3. The paper says it is 224x224x3, but for the math to come out right(The original paper said different numbers, but Andrej Karpathy, the former head of computer vision at Tesla, said it should be 227×227×3 (he said Alex didn't describe why he put 224×224×3)). So, we reshape the image size to 227x227x3.

In [5]:
# x_train = tf.pad(x_train, [[0, 0], [98, 97], [98, 97]]) / 255
# x_train = tf.expand_dims(x_train, axis = -1)
# x_train = tf.repeat(x_train, 3, axis = -1)
# x_train = tf.image.resize(x_train, (255, 255))

# Colab crashes due to RAM constraints

In [6]:
x_train = tf.pad(x_train, [[0, 0], [2,2], [2,2]])/255
x_test = tf.pad(x_test, [[0, 0], [2,2], [2,2]])/255
x_train = tf.expand_dims(x_train, axis=3)
x_test = tf.expand_dims(x_test, axis=3)
x_train = tf.repeat(x_train, 3, axis=3)
x_test = tf.repeat(x_test, 3, axis=3)
x_val = x_train[-2000:, :]
y_val = y_train[-2000:]
x_train = x_train[:-2000, :]
y_train = y_train[:-2000]

In [7]:
model = models.Sequential()

Using Bilinear Interpolation to resize the image

In [10]:
model.add(layers.experimental.preprocessing.Resizing(224, 224, interpolation="bilinear", input_shape=x_train.shape[1:]))

# 227x227x3 is now converted to 55x55x96 using
model.add(layers.Conv2D(96, 11, strides = 4, padding = "same"))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.Activation('relu'))

# max pooling layer applied on 55x55x96 with 3x3 filter and stride = 2 to obtain an img of 27x27x96
model.add(layers.MaxPooling2D(3, strides = 2))

# 'same' convolution with 256 5x5 filers to get img of size 27x27x256
model.add(layers.Conv2D(256, 5, strides = 4, padding = "same"))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.Activation('relu'))

# max pooling layer applied on 27x27x256 with 3x3 filter and stride = 2 to obtain an img of 13x13x256
model.add(layers.MaxPooling2D(3, strides = 2))

# 2 layers of 'same' convolution with 384 3x3 filters which makes the dim of the img to 13x13x384
model.add(layers.Conv2D(384, 3, strides = 4, padding = "same"))
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(384, 3, strides = 4, padding = "same"))
model.add(layers.Activation('relu'))

# one 'same' convolution with